### **Connecting with Drive**

In [1]:
#Reading the training data Subject
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Importing All needed libraries**

In [2]:
#Importing all needed libraries
import pandas as pd
import numpy as np #Matric math
import tensorflow as tf #ML
from tensorflow.python.framework import ops
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import sys
import os
from scipy.io import loadmat
from scipy.io import loadmat
ops.reset_default_graph()

tf.compat.v1.reset_default_graph() #Clearning cache
sess=tf.compat.v1.InteractiveSession()




### **Define Task**

In [3]:


def split_task_data(data, sequence_length=15, n_features=96, split_ratio=0.8):
    Z = np.array(data[:, :n_features], dtype=np.float32)  # Ensure correct dtype
    X = np.array(data[:, n_features:n_features+22], dtype=np.float32)  # Ensure correct slicing

    n_sequences = len(Z) - sequence_length + 1
    inputs = np.zeros((n_sequences, sequence_length, n_features), dtype=np.float32)
    outputs = np.zeros((n_sequences, X.shape[1]), dtype=np.float32)

    for j in range(n_sequences):
        inputs[j] = Z[j:j + sequence_length]
        outputs[j] = X[j + sequence_length - 1]

    split_index = int(len(inputs) * split_ratio)
    return inputs[:split_index], outputs[:split_index], inputs[split_index:], outputs[split_index:]


# Prepare tasks for MAML
def prepare_tasks(subjects, base_path, sequence_length=15, n_features=96):
    tasks = []
    for subject in subjects:
        data_path = f"{base_path}/S{subject}_E1.mat"
        subject_data = loadmat(data_path)["Data"]
        print(f"Shape of subject {subject}: {subject_data.shape}")
        support_x, support_y, query_x, query_y = split_task_data(subject_data, sequence_length, n_features)
        print(f"Shape of support_x for subject {subject}: {support_x.shape}")
        print(f"Shape of support_y for subject {subject}: {support_y.shape}")
        print(f"Shape of query_x for subject {subject}: {query_x.shape}")
        print(f"Shape of query_y for subject {subject}: {query_y.shape}")
        tasks.append((support_x, support_y, query_x, query_y))

    return tasks



### **build model**

In [4]:
from tensorflow.keras import layers, Model, Input

def build_model(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features))

    # CNN Layers
    x = layers.Conv1D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    x = layers.Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    x = layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    # LSTM Layers
    x = layers.Bidirectional(layers.LSTM(500, return_sequences=True))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Bidirectional(layers.LSTM(500))(x)
    x = layers.Dropout(0.2)(x)

    # Fully Connected Layer
    outputs = layers.Dense(output_dim, activation='linear')(x)
    return Model(inputs, outputs)


def build_deep_cnn_lstm(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features))

    # CNN Block 1
    x = layers.Conv1D(64, 5, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    skip1 = x  # Save for later residual connection

    # CNN Block 2
    x = layers.Conv1D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    skip2 = x  # Save for later residual connection

    # CNN Block 3 with residual
    x = layers.Conv1D(256, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # 🔥 Fix: Ensure `skip2` has the same shape as `x` before addition
    skip2 = layers.Conv1D(256, 1, padding='same', activation='linear')(skip2)  # 🔹 Match dimensions
    x = layers.add([x, skip2])  # Now both have (batch_size, timesteps, 256)

    # LSTM Block
    x = layers.Bidirectional(layers.LSTM(512, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(256))(x)

    # Dense Block
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(output_dim, activation='linear')(x)

    return Model(inputs, outputs)


def build_lstm_attention(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features))

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(inputs)

    # Attention Mechanism
    query = layers.Dense(256)(x)
    value = layers.Dense(256)(x)
    attention = layers.Attention()([query, value])

    # Temporal Pooling
    x = layers.GlobalAveragePooling1D()(attention)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(output_dim, activation='linear')(x)
    return Model(inputs, outputs)

def build_gru_model(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features), dtype=tf.float32)

    x = layers.Bidirectional(layers.GRU(256, return_sequences=True, dtype=tf.float32))(inputs)
    x = layers.Bidirectional(layers.GRU(128, dtype=tf.float32))(x)

    gate = layers.Dense(256, activation='sigmoid', dtype=tf.float32)(x)
    x = layers.Lambda(lambda tensors: tf.multiply(tf.cast(tensors[0], tf.float32),
                                                   tf.cast(tensors[1], tf.float32)))([x, gate])

    outputs = layers.Dense(output_dim, activation='linear', dtype=tf.float32)(x)
    return Model(inputs, outputs)


def build_time_distributed_model(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features))

    # Time Distributed Processing
    x = layers.TimeDistributed(layers.Dense(128, activation='relu'))(inputs)
    x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)

    # Temporal Processing
    x = layers.LSTM(128, return_sequences=True)(x)
    x = layers.GlobalMaxPooling1D()(x)

    outputs = layers.Dense(output_dim, activation='linear')(x)
    return Model(inputs, outputs)

def build_cnn_attention_hybrid(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features), dtype=tf.float32)  # Ensure input is float32

    # CNN Feature Extraction
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(128, 3, padding='same', activation='relu')(x)

    # Fix: Use Lambda layer instead of `tf.cast`
    x = layers.Lambda(lambda t: tf.cast(t, tf.float32))(x)  # Correct way

    # Multi-head Attention
    x = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)

    # Pooling and FC
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(256, activation='relu')(x)

    outputs = layers.Dense(output_dim, activation='linear')(x)
    return Model(inputs, outputs)




### **Training Model framework**

In [5]:
def inner_loop(model, support_x, support_y, loss_fn, inner_lr):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(support_x, tf.float32), training=True)
        loss = loss_fn(tf.cast(support_y, tf.float32), predictions)

    grads = tape.gradient(loss, model.trainable_variables)

    # Ensure gradients and variables are both float32 before update
    grads = [tf.cast(g, tf.float32) for g in grads]

    for var, g in zip(model.trainable_variables, grads):
        var.assign_sub(inner_lr * g)

    return model.trainable_variables, loss


def maml_step(model, tasks, loss_fn, meta_optimizer, inner_lr):
    meta_grads = [tf.zeros_like(var, dtype=tf.float32) for var in model.trainable_variables]

    for support_x, support_y, query_x, query_y in tasks:
        updated_weights, _ = inner_loop(model, support_x, support_y, loss_fn, inner_lr)

        with tf.GradientTape() as tape:
            query_predictions = model(tf.cast(query_x, tf.float32), training=True)
            query_loss = loss_fn(tf.cast(query_y, tf.float32), query_predictions)

        task_grads = tape.gradient(query_loss, model.trainable_variables)

        # ✅ Ensure gradients are not None before updating
        task_grads = [g if g is not None else tf.zeros_like(var, dtype=tf.float32)
                      for g, var in zip(task_grads, model.trainable_variables)]

        meta_grads = [meta_grad + task_grad for meta_grad, task_grad in zip(meta_grads, task_grads)]

    meta_grads = [grad / len(tasks) for grad in meta_grads]  # Normalize gradients
    meta_optimizer.apply_gradients(zip(meta_grads, model.trainable_variables))


### **Train Model**

In [6]:
import tensorflow as tf

def pearson_mse_loss(y_true, y_pred, alpha=0.1):
    """
    Computes a hybrid loss function that:
    - Maximizes Pearson Correlation Coefficient (CC)
    - Minimizes Mean Squared Error (MSE)

    Parameters:
    - y_true: Actual values
    - y_pred: Predicted values
    - alpha: Weighting factor for Pearson CC loss (adjustable)

    Returns:
    - Combined hybrid loss (weighted sum of CC loss and MSE loss)
    """
    # Compute Pearson CC component
    y_true_mean = tf.reduce_mean(y_true, axis=0)
    y_pred_mean = tf.reduce_mean(y_pred, axis=0)

    numerator = tf.reduce_sum((y_true - y_true_mean) * (y_pred - y_pred_mean), axis=0)
    denominator = tf.sqrt(tf.reduce_sum((y_true - y_true_mean) ** 2, axis=0)) * \
                  tf.sqrt(tf.reduce_sum((y_pred - y_pred_mean) ** 2, axis=0))

    correlation = numerator / (denominator + 1e-8)  # Avoid division by zero
    cc_loss = -tf.reduce_mean(correlation)  # Negative since we want to maximize CC

    # Compute MSE component
    mse_loss_fn = tf.keras.losses.MeanSquaredError()
    mse_loss = mse_loss_fn(y_true, y_pred)

    # Combine losses with weighting factor alpha
    return mse_loss + alpha * cc_loss



# MAML Training Loop
def maml_train(model, tasks, epochs, inner_lr, meta_lr):
    loss_fn = lambda y_true, y_pred: pearson_mse_loss(y_true, y_pred, alpha=0.1)
    meta_optimizer = tf.keras.optimizers.Adam(meta_lr)

    for epoch in range(epochs):
        maml_step(model, tasks, loss_fn, meta_optimizer, inner_lr)
        print(f"Epoch {epoch + 1}/{epochs} completed")

###############   MAML +Ensemble learning ###########

def maml_train_ensemble(num_models, tasks, epochs, inner_lr, meta_lr):
    """
    Train multiple MAML models (ensemble members) on the given tasks.

    Parameters:
    - num_models: Number of models in the ensemble.
    - tasks: List of tasks for meta-training.
    - epochs: Number of meta-training epochs.
    - inner_lr: Inner-loop learning rate.
    - meta_lr: Meta-learning rate for the outer loop.

    Returns:
    - ensemble: List of trained models.
    """
    ensemble = []

    for i in range(num_models):
        print(f"Training model {i + 1}/{num_models}...")
        model = build_model()  # Use the CNN-LSTM model
        maml_train(model, tasks, epochs, inner_lr, meta_lr)  # Meta-train the model
        ensemble.append(model)

    return ensemble

def maml_train_ensemble_diverse(ensemble, tasks, epochs, inner_lr, meta_lr):
    for model in ensemble:
        print(f"\nTraining {model.name}...")
        maml_train(model, tasks, epochs, inner_lr, meta_lr)
    return ensemble

def create_diverse_ensemble(num_models=5):
    architectures = [
        build_model,
        build_deep_cnn_lstm,
        build_lstm_attention,
        build_gru_model,
        build_time_distributed_model,
        build_cnn_attention_hybrid
    ]

    ensemble = []
    for i in range(num_models):
        # Cycle through different architectures
        model_builder = architectures[i % len(architectures)]
        model = model_builder()
        print(f"Initialized model {i+1} with {model_builder.__name__}")
        ensemble.append(model)

    return ensemble




### **Testing**

In [7]:

def fine_tune_ensemble(ensemble, support_x, support_y, loss_fn, inner_lr, fine_tune_steps=1, batch_size=512):
    """
    Fine-tune each model in the ensemble using support data.

    Parameters:
    - ensemble: List of models.
    - support_x, support_y: Support set for fine-tuning.
    - loss_fn: Loss function (should be Pearson CC loss).
    - inner_lr: Learning rate for fine-tuning.
    - fine_tune_steps: Number of fine-tuning steps.
    - batch_size: Batch size for training.

    Returns:
    - fine_tuned_weights: Updated model weights after fine-tuning.
    """
    fine_tuned_weights = []

    for i, model in enumerate(ensemble):
        print(f"Fine-tuning model {i + 1}/{len(ensemble)}...")

        dataset = tf.data.Dataset.from_tensor_slices((support_x, support_y)).batch(batch_size)

        for _ in range(fine_tune_steps):
            for batch_x, batch_y in dataset:
                updated_weights, _ = inner_loop(model, batch_x, batch_y, loss_fn, inner_lr)

        # Apply fine-tuned weights back to the model
        for var, w in zip(model.trainable_variables, updated_weights):
            var.assign(w)

        fine_tuned_weights.append(updated_weights)

    return fine_tuned_weights

def ensemble_predict_weighted_performance(ensemble, fine_tuned_weights, query_x, val_x, val_y):
    """
    Compute ensemble predictions using weighted averaging based on Pearson CC,
    but using validation data instead of test data.

    Parameters:
    - ensemble: List of trained models.
    - fine_tuned_weights: Fine-tuned weights for each model.
    - query_x: Input features for the test set.
    - val_x: Validation input features (used to compute weights).
    - val_y: Validation ground truth (used to compute weights).

    Returns:
    - weighted_predictions: Final ensemble predictions for the test set.
    """
    all_predictions = []
    cc_values = []

    for i, (model, weights) in enumerate(zip(ensemble, fine_tuned_weights)):
        print(f"Computing weight for model {i + 1}/{len(ensemble)}...")

        # Apply fine-tuned weights before predicting
        for var, w in zip(model.trainable_variables, weights):
            var.assign(w)

        # Predict on validation data, NOT test data
        val_predictions = model(val_x, training=False)
        all_predictions.append(model(query_x, training=False))  # Still predict on test set

        # Compute Pearson CC based on validation data
        cc = np.array([
            pearsonr(val_y[:, i], val_predictions[:, i])[0] for i in range(val_y.shape[1])
        ])
        cc_values.append(cc)

    all_predictions = np.array(all_predictions)  # Shape: (num_models, batch_size, output_dim)
    cc_values = np.array(cc_values)  # Shape: (num_models, output_dim)

    # Normalize correlation values as weights
    cc_values = np.maximum(cc_values, 0)  # Remove negative correlations (set to zero)
    weights = cc_values / (np.sum(cc_values, axis=0, keepdims=True) + 1e-8)  # Avoid division by zero

    # Weighted average using CC-based weights
    weighted_predictions = np.sum(all_predictions * weights[:, None, :], axis=0)

    print(f"Model Weights (based on validation correlation coefficient): {weights}")
    return weighted_predictions





### **Plot the result**

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


def normalize_predictions(predictions):
    mean = np.mean(predictions, axis=0)
    std = np.std(predictions, axis=0) + 1e-8  # Avoid division by zero
    return (predictions - mean) / std
def scale_predictions(predictions):
    min_val = np.min(predictions, axis=0)
    max_val = np.max(predictions, axis=0)
    return (predictions - min_val) / (max_val - min_val)
def smooth_predictions(predictions, window_size=5):
    smoothed = np.zeros_like(predictions)
    for i in range(predictions.shape[1]):  # Loop through each DOF
        smoothed[:, i] = np.convolve(predictions[:, i], np.ones(window_size)/window_size, mode='same')
        smoothed[:, i] = np.convolve(predictions[:, i], np.ones(window_size)/window_size, mode='same')
    return smoothed

def evaluate_and_plot(model, updated_weights, query_x, query_y, apply_smoothing=True, apply_normalization=True,apply_Scale=True):
    """
    Evaluate the model for each DOF, calculate CC, and plot actual vs. normalized or smoothed predictions.

    Parameters:
    - model: The trained model.
    - updated_weights: Fine-tuned weights for the task.
    - query_x: Input features for the query set.
    - query_y: Ground truth outputs for the query set.
    - apply_smoothing: Whether to smooth predictions.
    - apply_normalization: Whether to normalize predictions.

    Returns:
    - correlations: List of CC values for each DOF.
    - avg_correlation: Average CC across all DOFs.
    """
    # Predict using the updated weights
    query_predictions = model(query_x, training=False)

    # Normalize or smooth predictions

    if apply_smoothing:
        query_predictions = smooth_predictions(query_predictions)
    if apply_normalization:
        query_predictions = normalize_predictions(query_predictions)
    if apply_Scale:
        query_predictions = scale_predictions(query_predictions)

    # Initialize lists for storing CC values
    correlations = []

    # Loop through each DOF (dimension)
    for i in range(query_y.shape[1]):
        # Calculate Pearson Correlation Coefficient
        cc_value, _ = pearsonr(query_y[:, i], query_predictions[:, i])
        correlations.append(cc_value)

        # Plot Actual vs Predicted for the current DOF
        plt.figure(figsize=(10, 6))
        plt.plot(query_y[:, i], label='Actual', color='blue')
        plt.plot(query_predictions[:, i], label='Predicted', color='red')

        # Add title and labels
        plt.title(f'Degree of Freedom {i + 1}: CC = {cc_value:.2f}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()

    # Compute the average CC across all DOFs
    avg_correlation = np.mean(correlations)
    print(f"Average Pearson Correlation Coefficient: {avg_correlation:.2f}")

    return correlations, avg_correlation

def evaluate_ensemble(ensemble_predictions, query_y, apply_smoothing=True, apply_normalization=True, apply_scale=True):
    """
    Evaluate ensemble predictions and plot results.

    Parameters:
    - ensemble_predictions: Predictions from the ensemble.
    - query_y: Ground truth for the query set.
    - apply_smoothing: Whether to smooth predictions.
    - apply_normalization: Whether to normalize predictions.
    - apply_scale: Whether to scale predictions.

    Returns:
    - correlations: List of CC values for each DOF.
    - avg_correlation: Average CC across all DOFs.
    """
    # Process predictions
    if apply_smoothing:
        ensemble_predictions = smooth_predictions(ensemble_predictions)
    if apply_normalization:
        ensemble_predictions = normalize_predictions(ensemble_predictions)
    if apply_scale:
        ensemble_predictions = scale_predictions(ensemble_predictions)

    # Initialize lists for storing CC values
    correlations = []

    # Loop through each DOF (dimension)
    for i in range(query_y.shape[1]):
        # Calculate Pearson Correlation Coefficient
        cc_value, _ = pearsonr(query_y[:, i], ensemble_predictions[:, i])
        correlations.append(cc_value)

        # Plot Actual vs Predicted for the current DOF
        plt.figure(figsize=(10, 6))
        plt.plot(query_y[:, i], label='Actual', color='blue')
        plt.plot(ensemble_predictions[:, i], label='Predicted', color='red')

        # Add title and labels
        plt.title(f'Degree of Freedom {i + 1}: CC = {cc_value:.2f}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()

    # Compute the average CC across all DOFs
    avg_correlation = np.mean(correlations)
    print(f"Average Pearson Correlation Coefficient: {avg_correlation:.2f}")

    return correlations, avg_correlation


def diverse_ensemble_predict(ensemble, query_x):
    predictions = []
    for model in ensemble:
        # Use different preprocessing for different models if needed
        pred = model.predict(query_x)
        predictions.append(pred)

    # Weighted averaging (could implement more sophisticated fusion)
    return np.mean(predictions, axis=0)


## **Methods for saving the data**

In [9]:
# Save data in Excel file
def savedata(i, correlations):
    """
    Save the correlations data in an Excel file at the specified row.

    Parameters:
    - i: Integer representing the subject number (e.g., 5 for row name 'S5').
    - correlations: List of correlation values to save in the row.
    """
    import openpyxl
    from google.colab import drive

    # Mount Google Drive
    drive.mount('/content/drive')

    # Path to the workbook
    workbook_path = '/content/drive/My Drive/Colab Notebooks/dataset/Ninapro MAML-8features- 6models_Subjectindependant.xlsx'

    # Load the existing workbook
    wb = openpyxl.load_workbook(workbook_path)

    # Select the active sheet
    sheet = wb.active

    # Dynamically set the row name based on 'i'
    row_name = f'S{i}'  # Example: 'S5' if i=5

    # Find the row with the specified row name
    target_row = None
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row):
        if row[0].value == row_name:
            target_row = row[0].row
            break

    if target_row is None:
        print(f"Error: Row with name '{row_name}' not found in the sheet.")
        return

    # Write the data to the found row, starting from the second column
    for col, value in enumerate(correlations, start=2):  # Start at column 2 to skip the first column for the label
        sheet.cell(row=target_row, column=col, value=value)

    # Save the workbook
    wb.save(workbook_path)
    print(f"Data successfully saved for row '{row_name}'.")



In [10]:
import random
def process_subject(test_subject, ensemble, base_path, train_subjects, fine_tune_subjects=5, val_ratio=0.2):
    print(f"Processing Test Subject: S{test_subject}")

    test_data = loadmat(f"{base_path}/S{test_subject}_E1.mat")["Data"]

    Z = test_data[:, :96]
    X = test_data[:, 96:118]

    sequence_length = 15
    n_sequences = len(Z) - sequence_length + 1

    query_x = np.zeros((n_sequences, sequence_length, 96), dtype=np.float32)
    query_y = np.zeros((n_sequences, X.shape[1]), dtype=np.float32)

    for j in range(n_sequences):
        query_x[j] = Z[j:j + sequence_length]
        query_y[j] = X[j + sequence_length - 1]

    # Ensure test subject is NEVER in fine-tuning set
    fine_tune_subjects_list = random.sample([s for s in train_subjects if s != test_subject], fine_tune_subjects)
    print(f"Fine-tuning with {fine_tune_subjects_list}")

    support_x_list, support_y_list = [], []
    val_x_list, val_y_list = [], []

    for subject in fine_tune_subjects_list:
        train_data = loadmat(f"{base_path}/S{subject}_E1.mat")["Data"]
        support_x, support_y, val_x, val_y = split_task_data(train_data, split_ratio=1 - val_ratio)
        support_x_list.append(support_x)
        support_y_list.append(support_y)
        val_x_list.append(val_x)
        val_y_list.append(val_y)

    support_x = np.concatenate(support_x_list, axis=0)
    support_y = np.concatenate(support_y_list, axis=0)
    val_x = np.concatenate(val_x_list, axis=0)
    val_y = np.concatenate(val_y_list, axis=0)

    # Ensure fine-tuning uses the correct loss
    fine_tuned_weights = fine_tune_ensemble(ensemble, support_x, support_y, pearson_mse_loss, 0.01)

    # Compute ensemble weights using validation data, NOT test data
    ensemble_predictions = ensemble_predict_weighted_performance(ensemble, fine_tuned_weights, query_x, val_x, val_y)

    return evaluate_ensemble(ensemble_predictions, query_y, apply_smoothing=True, apply_normalization=False, apply_scale=True)


### **Main method**

In [11]:

import os

if __name__ == "__main__":
    base_path = '/content/drive/My Drive/Colab Notebooks/dataset wstimi/8 Features'
    all_subjects = list(range(1, 41))  # Subjects 1 to 40

    # Start resuming from test subject 10
    start_subject = 18

    for test_subject in all_subjects:
        # Skip subjects that were already processed
        if test_subject < start_subject:
            continue  # Skip to the starting subject

        print(f"Processing S{test_subject}")

        # 1. Load data with explicit dtype
        train_subjects = [s for s in all_subjects if s != test_subject]
        tasks = prepare_tasks(train_subjects, base_path)

        # 2. Create ensemble with dtype-consistent models
        ensemble = create_diverse_ensemble(num_models=6)

        # 3. Train with dtype-safe settings
        trained_ensemble = maml_train_ensemble_diverse(
            ensemble, tasks, epochs=100, inner_lr=0.01, meta_lr=0.001
        )

        # 4. Process subject with dtype-controlled data
        correlations, _ = process_subject(test_subject, ensemble, base_path, train_subjects)

        # Save the results
        savedata(test_subject, correlations)



Output hidden; open in https://colab.research.google.com to view.